In [1]:
import pickle as pkl
import os
import pandas as pd

path = os.getcwd()
data_name = 'SMD'
data_file = os.path.join(path, 'processed', data_name)
train_data_ = open(os.path.join(data_file, 'machine-1-1_train.pkl'),'rb')
train_data = pd.DataFrame(pkl.load(train_data_))
test_data_ = open(os.path.join(data_file, 'machine-1-1_test.pkl'),'rb')
test_data = pd.DataFrame(pkl.load(test_data_))

In [2]:
test_data_label_ = open(os.path.join(data_file, 'machine-1-1_test_label.pkl'),'rb')
test_data_label = pd.DataFrame(pkl.load(test_data_label_),columns=['label'])

In [3]:
anomaly_num = len(test_data_label[test_data_label.label==1])
anomaly_num / len(test_data_label)

0.09459601811861372

In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_train_data = pd.DataFrame(scaler.fit_transform(train_data))
normalized_test_data = pd.DataFrame(scaler.fit_transform(test_data))
normalized_test_data

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,0.065217,0.065678,0.068156,0.073256,0.0,0.925287,0.269886,0.0,0.031081,0.000000,...,0.0,0.004317,0.068036,0.048893,0.000386,0.000023,0.064432,0.064500,0.0,0.0
1,0.076087,0.080508,0.073742,0.075582,0.0,0.922414,0.270833,0.0,0.031081,0.000122,...,0.0,0.004317,0.070020,0.050437,0.000386,0.000011,0.065228,0.065224,0.0,0.0
2,0.065217,0.064619,0.069273,0.073256,0.0,0.919540,0.270833,0.0,0.030940,0.000366,...,0.0,0.004317,0.069684,0.055069,0.000386,0.000034,0.067111,0.067178,0.0,0.0
3,0.076087,0.048729,0.061452,0.069768,0.0,0.919540,0.268940,0.0,0.027250,0.000244,...,0.0,0.005756,0.073253,0.051467,0.000000,0.000023,0.066676,0.066744,0.0,0.0
4,0.076087,0.051907,0.060335,0.069768,0.0,0.925287,0.269886,0.0,0.030940,0.000244,...,0.0,0.004317,0.070932,0.051467,0.000386,0.000011,0.066604,0.066671,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28474,0.065217,0.051907,0.045810,0.046512,0.0,0.896551,0.252841,0.0,0.043571,0.000244,...,0.0,0.027339,0.046733,0.040144,0.000000,0.000011,0.042931,0.043000,0.0,0.0
28475,0.054347,0.025424,0.036871,0.043024,0.0,0.893678,0.252841,0.0,0.032501,0.000000,...,0.0,0.027339,0.047438,0.048893,0.000000,0.000045,0.046550,0.046619,0.0,0.0
28476,0.054347,0.080508,0.048044,0.046512,0.0,0.896551,0.253788,0.0,0.026114,0.000611,...,0.0,0.027339,0.046797,0.040144,0.000386,0.000034,0.043003,0.043000,0.0,0.0
28477,0.054347,0.056144,0.045810,0.045350,0.0,0.890805,0.252841,0.0,0.033210,0.000122,...,0.0,0.027339,0.041884,0.043232,0.000000,0.000034,0.039890,0.039959,0.0,0.0


In [5]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=5)
pca_train_data = pca.fit_transform(normalized_train_data)
pca_test_data = pca.fit_transform(normalized_test_data)

print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

[0.6472448  0.23495732 0.0381447  0.01958584 0.01584373]
0.9557764139026403
[67.19859  40.487446 16.313345 11.689529 10.513678]


## Base learner

In [6]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pyod is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn import metrics

In [7]:
def pred_labels(y_test, test_scores):
    fpr, tpr, thresholds = roc_curve(y_test, test_scores)
    cutoff = thresholds[np.argmax(tpr - fpr)]
    pred_label = []
    for each in test_scores:
        if each > cutoff:
            pred_label.append(1)
        else:
            pred_label.append(0)    
    return pred_label

In [8]:
# X_train = normalized_test_data
# X_test = normalized_test_data
X_train = pca_test_data
X_test = pca_test_data
y_train = test_data_label
y_test = test_data_label

In [9]:
def base_models():
    # Define nine outlier detection tools to be compared
    random_state = np.random.RandomState(42)

    #df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
    #              'ABOD', 'CBLOF', 'HBOS', 'IForest', 'KNN', 'LOF', 'MCD',
    #              'OCSVM']
    df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
                  'IForest', 'KNN', 'LOF', 'OCSVM']

    pr_df = pd.DataFrame(columns=df_columns)
    re_df = pd.DataFrame(columns=df_columns)
    f1_df = pd.DataFrame(columns=df_columns)
    roc_df = pd.DataFrame(columns=df_columns)
    train_time_df = pd.DataFrame(columns=df_columns)
    test_time_df = pd.DataFrame(columns=df_columns)
    label_df = pd.DataFrame(columns=df_columns[4:])

    outliers_fraction = np.count_nonzero(y_train) / len(y_train)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    pr_list = ['Monitor_data', X_train.shape[0], X_train.shape[1], outliers_percentage]
    re_list = ['Monitor_data', X_train.shape[0], X_train.shape[1], outliers_percentage]
    f1_list = ['Monitor_data', X_train.shape[0], X_train.shape[1], outliers_percentage]
    roc_list = ['Monitor_data', X_train.shape[0], X_train.shape[1], outliers_percentage]
    train_time_list = ['Monitor_data', X_train.shape[0], X_train.shape[1], outliers_percentage]
    test_time_list = ['Monitor_data', X_train.shape[0], X_train.shape[1], outliers_percentage]
    label_list = []

    classifiers = {
        #'Angle-based Outlier Detector (ABOD)': ABOD(
        #    contamination=outliers_fraction),
        #    'Cluster-based Local Outlier Factor': CBLOF(
        #        contamination=outliers_fraction, check_estimator=False,
        #        random_state=random_state),
        #    'Histogram-base Outlier Detection (HBOS)': HBOS(
        #        contamination=outliers_fraction),
            'Isolation Forest': IForest(contamination=outliers_fraction,
                                        random_state=random_state),
            'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
            'Local Outlier Factor (LOF)': LOF(
                contamination=outliers_fraction),
        #   'Minimum Covariance Determinant (MCD)': MCD(
        #        contamination=outliers_fraction, random_state=random_state),
            'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
            #'Principal Component Analysis (PCA)': PCA(
            #    contamination=outliers_fraction, random_state=random_state),
        }

    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train)
        t1 = time()
        test_scores = clf.decision_function(X_test)
        t2 = time()
        mean_score = np.nanmean(test_scores)
        test_scores[np.isnan(test_scores)] = mean_score
        train_duration = round(t1 - t0, ndigits=4)
        test_duration = round(t2-t1, ndigits=4)
        train_time_list.append(train_duration)
        test_time_list.append(test_duration)
        #print(test_scores.tolist())
        pred_label = pred_labels(y_test, test_scores)
        pr = round(metrics.precision_score(y_test, pred_label, average='macro'), ndigits=4)
        re = round(metrics.recall_score(y_test, pred_label, average='macro'), ndigits=4)
        f1 = round(f1_score(y_test, pred_label, average='macro'), ndigits=4)
        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        print('{clf_name} ROC:{roc}, f1 score:{f1}, '
              'execution time: {duration}s'.format(clf_name=clf_name, roc=roc, f1=f1, duration=train_duration))

        pr_list.append(pr)
        re_list.append(re)
        f1_list.append(f1)
        roc_list.append(roc)
        label_list.append(pred_label)

    temp_df = pd.DataFrame(train_time_list).transpose()
    temp_df.columns = df_columns
    train_time_df = pd.concat([train_time_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(test_time_list).transpose()
    temp_df.columns = df_columns
    test_time_df = pd.concat([test_time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(f1_list).transpose()
    temp_df.columns = df_columns
    f1_df = pd.concat([f1_df, temp_df], axis=0)

    temp_df = pd.DataFrame(label_list).transpose()
    temp_df.columns = df_columns[4:]
    label_df = pd.concat([label_df, temp_df], axis=0)

    temp_df = pd.DataFrame(pr_list).transpose()
    temp_df.columns = df_columns
    pr_df = pd.concat([pr_df, temp_df], axis=0)

    temp_df = pd.DataFrame(re_list).transpose()
    temp_df.columns = df_columns
    re_df = pd.concat([re_df, temp_df], axis=0)
    return train_time_df, test_time_df, pr_df, re_df, f1_df, roc_df

In [10]:
df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc',
              'IForest', 'KNN', 'LOF', 'OCSVM']
t_pr_df = pd.DataFrame(columns=df_columns)
t_re_df = pd.DataFrame(columns=df_columns)
t_f1_df = pd.DataFrame(columns=df_columns)
t_roc_df = pd.DataFrame(columns=df_columns)
t_train_time_df = pd.DataFrame(columns=df_columns)
t_test_time_df = pd.DataFrame(columns=df_columns)

for i in range(10):
    print(i)
    train_time_e, test_time_e, pr_e, re_e, f1_e, roc_e = base_models()
    t_pr_df = pd.concat([t_pr_df, pr_e], axis=0, ignore_index=True)
    t_re_df = pd.concat([t_re_df, re_e], axis=0, ignore_index=True)
    t_f1_df = pd.concat([t_f1_df, f1_e], axis=0, ignore_index=True)
    t_roc_df = pd.concat([t_roc_df, roc_e], axis=0, ignore_index=True)
    t_train_time_df = pd.concat([t_train_time_df, train_time_e], axis=0, ignore_index=True)
    t_test_time_df = pd.concat([t_test_time_df, test_time_e], axis=0, ignore_index=True)

0
Isolation Forest ROC:0.8937, f1 score:0.7473, execution time: 1.1137s
K Nearest Neighbors (KNN) ROC:0.8104, f1 score:0.5713, execution time: 0.3077s
Local Outlier Factor (LOF) ROC:0.5495, f1 score:0.5468, execution time: 0.5232s
One-class SVM (OCSVM) ROC:0.8381, f1 score:0.6047, execution time: 65.895s
1
Isolation Forest ROC:0.8937, f1 score:0.7473, execution time: 1.1387s
K Nearest Neighbors (KNN) ROC:0.8104, f1 score:0.5713, execution time: 0.314s
Local Outlier Factor (LOF) ROC:0.5495, f1 score:0.5468, execution time: 0.5312s
One-class SVM (OCSVM) ROC:0.8381, f1 score:0.6047, execution time: 66.2758s
2
Isolation Forest ROC:0.8937, f1 score:0.7473, execution time: 1.1536s
K Nearest Neighbors (KNN) ROC:0.8104, f1 score:0.5713, execution time: 0.3069s
Local Outlier Factor (LOF) ROC:0.5495, f1 score:0.5468, execution time: 0.6162s
One-class SVM (OCSVM) ROC:0.8381, f1 score:0.6047, execution time: 67.4297s
3
Isolation Forest ROC:0.8937, f1 score:0.7473, execution time: 1.1112s
K Nearest

In [11]:
print('Train time complexity')
learner = ['IForest', 'KNN', 'LOF', 'OCSVM']
mean_train_time = []
std_train_time = []
for each in learner:
    mean_train_time.append(round(t_train_time_df[each].mean(), ndigits=4))
    std_train_time.append(round(t_train_time_df[each].std(), ndigits=4))
print(mean_train_time)
print(std_train_time)
t_train_time_df

Train time complexity
[1.1237, 0.3069, 0.5366, 66.358]
[0.03, 0.013, 0.0306, 1.11]


,Data,#Samples,# Dimensions,Outlier Perc,IForest,KNN,LOF,OCSVM
0,Monitor_data,28479,5,9.4596,1.1137,0.3077,0.5232,65.895
1,Monitor_data,28479,5,9.4596,1.1387,0.314,0.5312,66.2758
2,Monitor_data,28479,5,9.4596,1.1536,0.3069,0.6162,67.4297
3,Monitor_data,28479,5,9.4596,1.1112,0.295,0.5142,67.6661
4,Monitor_data,28479,5,9.4596,1.0966,0.2948,0.5192,65.7885
5,Monitor_data,28479,5,9.4596,1.103,0.3017,0.5268,65.6665
6,Monitor_data,28479,5,9.4596,1.192,0.3399,0.5597,68.4949
7,Monitor_data,28479,5,9.4596,1.1048,0.3051,0.5305,65.1792
8,Monitor_data,28479,5,9.4596,1.1226,0.3019,0.5249,65.7865
9,Monitor_data,28479,5,9.4596,1.1007,0.3017,0.5203,65.3973


In [12]:
print('Test time complexity')
learner = ['IForest', 'KNN', 'LOF', 'OCSVM']
mean_test_time = []
std_test_time = []
for each in learner:
    mean_test_time.append(round(t_test_time_df[each].mean(), ndigits=4))
    std_test_time.append(round(t_test_time_df[each].std(), ndigits=4))
print(mean_test_time)
print(std_test_time)
t_test_time_df

Test time complexity
[0.4321, 1.9017, 0.4873, 33.5172]
[0.008, 0.0943, 0.0166, 1.2595]


,Data,#Samples,# Dimensions,Outlier Perc,IForest,KNN,LOF,OCSVM
0,Monitor_data,28479,5,9.4596,0.4301,1.8434,0.4803,32.8692
1,Monitor_data,28479,5,9.4596,0.436,1.9619,0.4907,34.4135
2,Monitor_data,28479,5,9.4596,0.4459,2.0145,0.5037,33.0317
3,Monitor_data,28479,5,9.4596,0.425,1.8424,0.4779,32.8424
4,Monitor_data,28479,5,9.4596,0.4222,1.8439,0.4798,32.8801
5,Monitor_data,28479,5,9.4596,0.4412,1.8461,0.4761,36.7175
6,Monitor_data,28479,5,9.4596,0.435,2.1079,0.5282,33.129
7,Monitor_data,28479,5,9.4596,0.4256,1.8678,0.478,33.9253
8,Monitor_data,28479,5,9.4596,0.4363,1.8548,0.4794,32.6995
9,Monitor_data,28479,5,9.4596,0.4235,1.8344,0.4791,32.6637


In [13]:
print('ROC Performance')
learner = ['IForest', 'KNN', 'LOF', 'OCSVM']
mean_roc = []
for each in learner:
    mean_roc.append(round(t_roc_df[each].mean(), ndigits=4))
print(mean_roc)
t_roc_df

ROC Performance
[0.8937, 0.8104, 0.5495, 0.8381]


,Data,#Samples,# Dimensions,Outlier Perc,IForest,KNN,LOF,OCSVM
0,Monitor_data,28479,5,9.4596,0.8937,0.8104,0.5495,0.8381
1,Monitor_data,28479,5,9.4596,0.8937,0.8104,0.5495,0.8381
2,Monitor_data,28479,5,9.4596,0.8937,0.8104,0.5495,0.8381
3,Monitor_data,28479,5,9.4596,0.8937,0.8104,0.5495,0.8381
4,Monitor_data,28479,5,9.4596,0.8937,0.8104,0.5495,0.8381
5,Monitor_data,28479,5,9.4596,0.8937,0.8104,0.5495,0.8381
6,Monitor_data,28479,5,9.4596,0.8937,0.8104,0.5495,0.8381
7,Monitor_data,28479,5,9.4596,0.8937,0.8104,0.5495,0.8381
8,Monitor_data,28479,5,9.4596,0.8937,0.8104,0.5495,0.8381
9,Monitor_data,28479,5,9.4596,0.8937,0.8104,0.5495,0.8381


In [14]:
print('Precision')
learner = ['IForest', 'KNN', 'LOF', 'OCSVM']
mean_pr = []
for each in learner:
    mean_pr.append(round(t_pr_df[each].mean(), ndigits=4))
print(mean_pr)
t_pr_df

Precision
[0.7087, 0.5905, 0.5425, 0.6126]


,Data,#Samples,# Dimensions,Outlier Perc,IForest,KNN,LOF,OCSVM
0,Monitor_data,28479,5,9.4596,0.7087,0.5905,0.5425,0.6126
1,Monitor_data,28479,5,9.4596,0.7087,0.5905,0.5425,0.6126
2,Monitor_data,28479,5,9.4596,0.7087,0.5905,0.5425,0.6126
3,Monitor_data,28479,5,9.4596,0.7087,0.5905,0.5425,0.6126
4,Monitor_data,28479,5,9.4596,0.7087,0.5905,0.5425,0.6126
5,Monitor_data,28479,5,9.4596,0.7087,0.5905,0.5425,0.6126
6,Monitor_data,28479,5,9.4596,0.7087,0.5905,0.5425,0.6126
7,Monitor_data,28479,5,9.4596,0.7087,0.5905,0.5425,0.6126
8,Monitor_data,28479,5,9.4596,0.7087,0.5905,0.5425,0.6126
9,Monitor_data,28479,5,9.4596,0.7087,0.5905,0.5425,0.6126


In [15]:
print('Recall')
learner = ['IForest', 'KNN', 'LOF', 'OCSVM']
mean_re = []
for each in learner:
    mean_re.append(round(t_re_df[each].mean(), ndigits=4))
print(mean_re)
t_re_df

Recall
[0.8257, 0.741, 0.5642, 0.7945]


,Data,#Samples,# Dimensions,Outlier Perc,IForest,KNN,LOF,OCSVM
0,Monitor_data,28479,5,9.4596,0.8257,0.741,0.5642,0.7945
1,Monitor_data,28479,5,9.4596,0.8257,0.741,0.5642,0.7945
2,Monitor_data,28479,5,9.4596,0.8257,0.741,0.5642,0.7945
3,Monitor_data,28479,5,9.4596,0.8257,0.741,0.5642,0.7945
4,Monitor_data,28479,5,9.4596,0.8257,0.741,0.5642,0.7945
5,Monitor_data,28479,5,9.4596,0.8257,0.741,0.5642,0.7945
6,Monitor_data,28479,5,9.4596,0.8257,0.741,0.5642,0.7945
7,Monitor_data,28479,5,9.4596,0.8257,0.741,0.5642,0.7945
8,Monitor_data,28479,5,9.4596,0.8257,0.741,0.5642,0.7945
9,Monitor_data,28479,5,9.4596,0.8257,0.741,0.5642,0.7945


In [16]:
print('F1 score')
learner = ['IForest', 'KNN', 'LOF', 'OCSVM']
mean_f1 = []
for each in learner:
    mean_f1.append(round(t_f1_df[each].mean(), ndigits=4))
print(mean_f1)
t_f1_df

F1 score
[0.7473, 0.5713, 0.5468, 0.6047]


,Data,#Samples,# Dimensions,Outlier Perc,IForest,KNN,LOF,OCSVM
0,Monitor_data,28479,5,9.4596,0.7473,0.5713,0.5468,0.6047
1,Monitor_data,28479,5,9.4596,0.7473,0.5713,0.5468,0.6047
2,Monitor_data,28479,5,9.4596,0.7473,0.5713,0.5468,0.6047
3,Monitor_data,28479,5,9.4596,0.7473,0.5713,0.5468,0.6047
4,Monitor_data,28479,5,9.4596,0.7473,0.5713,0.5468,0.6047
5,Monitor_data,28479,5,9.4596,0.7473,0.5713,0.5468,0.6047
6,Monitor_data,28479,5,9.4596,0.7473,0.5713,0.5468,0.6047
7,Monitor_data,28479,5,9.4596,0.7473,0.5713,0.5468,0.6047
8,Monitor_data,28479,5,9.4596,0.7473,0.5713,0.5468,0.6047
9,Monitor_data,28479,5,9.4596,0.7473,0.5713,0.5468,0.6047


# 